In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [11]:
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
model = Sequential()
model.add(BatchNormalization(input_shape=(64,64,3)))
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(5))
model.add(Activation("softmax"))

model.summary()

# We do not compile the model now!
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
activation_8 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_9 (Activation)    (None, 32, 32, 32)       

In [3]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
          continue
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            if file.endswith('jpg'):
                image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
                #image = preprocess_input(image)
                image = cv2.resize(image, (64,64))
                samples.append(image)
                labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('flower_photos')
print('loaded',len(samples),' samples')
print('classes',set(labels))

samples = samples / 255
# one-hot labels
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print("Labels shape",labels.shape)
labels = labels.astype(float)

loaded 3670  samples
classes {'daisy', 'sunflowers', 'tulips', 'roses', 'dandelion'}
Labels shape (3670, 5)


In [4]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels,random_state=42)
testSamples.shape

(918, 64, 64, 3)

In [12]:
from tensorflow.keras.losses import categorical_crossentropy
# opt = tf.keras.optimizers.Adam(learning_rate=0.05
#                                , decay=0.01    ## when decay is used the learning rate decreases!
#                               )
opt = tf.keras.optimizers.Adam(0.0001)
@tf.function
def step(tsamples, tlabels):
    with tf.GradientTape() as tape:
        predictions = model(tsamples)
        loss = categorical_crossentropy(tlabels,predictions)
    #print("loss",loss.numpy().mean())
    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
step_no=0

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
import time

EPOCHS = 50
for i in range(EPOCHS):
    print('=========== STEP ',step_no)
    step_no+=1
    start = time.time()  
    step(trainSamples,trainLabels)
    end = time.time()
    testResults = model.predict(testSamples)
    print(confusion_matrix(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
    #print(classification_report(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
    accuracy = accuracy_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))
    print(f'Accuracy: {accuracy:.2f} time',(end - start))
    trainResults = model.predict(trainSamples)
    trainAccuracy = accuracy_score(trainLabels.argmax(axis=1), trainResults.argmax(axis=1))
    print(f'Train accuracy: {trainAccuracy:.2f}')
print('Done')

=========== STEP  10
[[ 21 106   0  19  34]
 [  5 149   0  15  45]
 [  7  61   1  37  61]
 [  3  26   0  92  49]
 [  7  49   0  41  90]]
Accuracy: 0.38 time 4.102533340454102
Train accuracy: 0.42
=========== STEP  11
[[ 45  68   0  14  53]
 [ 18 115   0  15  66]
 [ 12  36   2  26  91]
 [  6  14   0  82  68]
 [ 16  28   1  39 103]]
Accuracy: 0.38 time 3.9815945625305176
Train accuracy: 0.42
=========== STEP  12
[[ 56  62   2  13  47]
 [ 27 104   6  13  64]
 [ 22  25  21  28  71]
 [  9  13   6  79  63]
 [ 18  24  12  37  96]]
Accuracy: 0.39 time 4.219901084899902
Train accuracy: 0.43
=========== STEP  13
[[ 61  68   7  13  31]
 [ 28 112  21  13  40]
 [ 25  31  46  26  39]
 [ 12  12  32  73  41]
 [ 20  23  45  30  69]]
Accuracy: 0.39 time 4.562351703643799
Train accuracy: 0.43
=========== STEP  14
[[ 60  78   5  15  22]
 [ 26 124  17  13  34]
 [ 24  41  45  28  29]
 [ 15  16  29  77  33]
 [ 21  29  44  31  62]]
Accuracy: 0.40 time 4.385646820068359
Train accuracy: 0.44
=========== STEP  1